# Ristiintaulukointi

In [1]:
import pandas as pd
df=pd.read_excel('http://www.elisanet.fi/taanila/data1.xlsx',sheetname='Data')

In [2]:
#Ristiintaulukointi sujuu crosstab-toiminnolla, margins=True tuottaa rivi- ja sarakesummat
df1=pd.crosstab(df.sukupuoli, df.koulutus,  margins=True)

#Sukupuolia on merkitty aineistossa numeroilla 1 ja 2; haluan niille oikeat nimet
df1.index=['Mies','Nainen','Yhteensä']

#Koulutusasteita on merkitty aineistossa numeroilla 1, 2, 3 ja 4; haluan niille oikeat nimet
df1.columns = ['Peruskoulu', '2. aste', 'Korkeakoulu','Ylempi korkeakoulu','Yhteensä']

df1

,Peruskoulu,2. aste,Korkeakoulu,Ylempi korkeakoulu,Yhteensä
Mies,22,23,15,2,62
Nainen,5,7,7,0,19
Yhteensä,27,30,22,2,81


In [3]:
#Jos muuttujan nimessä on välilyönti, niin se täytyy laittaa hakasulkuihin hipsujen/lainausmerkkien sisään
#normalize='columns' tuottaa lukumäärien sijasta prosentit sarakkeiden summista
df2=pd.crosstab(df['tyytyväisyys johtoon'], df.sukupuoli,  margins=True, normalize='columns')

df2.columns=['Mies','Nainen','Yhteensä']
df2.index=['Erittäin tyytymätön','Tyytymätön','Ei tyytymätön eikä tyytyväinen','Tyytyväinen','Erittäin tyytyväinen']

#Ilman muotoilua prosentit näkyisivät desimaalilukuina
df2.style.format('{:.1%}')

In [4]:
#Voin korvata muuttujan numeroarvot oikeilla nimillä myös itse aineistoon
df.sukupuoli.replace([1,2], ['Mies','Nainen'], inplace=True)
df.perhe.replace([1,2], ['Perheetön','Perheellinen'], inplace=True)

#Ristiintaulukointii voin lisät myös useampia muuttujia
df3=pd.crosstab(df['tyytyväisyys johtoon'],[df.sukupuoli,df.perhe], margins=True)

#Voin nimetä uudelleen rivi (index)- ja sarakeotsikoita (columns)
df3 = df3.rename(columns={'All': 'Yhteensä'})
df3=df3.rename(index={'All': 'Yhteensä'})

df3

sukupuoli                    Mies                 Nainen           Yhteensä
perhe                Perheellinen Perheetön Perheellinen Perheetön         
tyytyväisyys johtoon                                                       
1                               3         4            0         0        7
2                               8         7            1         0       16
3                              12        11            5         2       30
4                              10         5            7         1       23
5                               2         1            3         0        6
Yhteensä                       35        28           16         3       82

In [5]:
#Muodostan osa-aineiston df_osa, johon otan mukaan vain tietyt sarakkeet
#Viimeistään tästä kannattaa oppia nimeämään muuttujia lyhyemmin
columns=['sukupuoli','perhe','koulutus','tyytyväisyys johtoon', 'tyytyväisyys työtovereihin',
        'tyytyväisyys työympäristöön','tyytyväisyys palkkaan','tyytyväisyys työtehtäviin']
df_osa=pd.DataFrame(df,columns=columns)

#Käyn for-silmukalla läpi kaikki df_osa -dataframen muuttujat (sarakkeet)
#Laskeen jokaiselle muuttujalle ristiintaulukoinnin sukupuolen kanssa
for var in df_osa:
    
    #Tarkastan ettei var ole sukupuoli, koska en voi ristiintaulukoida muuttujaa itsensä kanssa
    if var!='sukupuoli':
        df4=pd.crosstab(df_osa[var],df_osa.sukupuoli)
        print(df4)

sukupuoli     Mies  Nainen
perhe                     
Perheellinen    35      16
Perheetön       28       3
sukupuoli  Mies  Nainen
koulutus               
1.0          22       5
2.0          23       7
3.0          15       7
4.0           2       0
sukupuoli             Mies  Nainen
tyytyväisyys johtoon              
1                        7       0
2                       15       1
3                       23       7
4                       15       8
5                        3       3
sukupuoli                   Mies  Nainen
tyytyväisyys työtovereihin              
2.0                            3       0
3.0                            9       7
4.0                           27       8
5.0                           23       4
sukupuoli                    Mies  Nainen
tyytyväisyys työympäristöön              
1                               9       0
2                               7       2
3                              27       3
4                              15       8
5    

In [6]:
#Seuraavassa lasken df_osa-dataframelle kaikki mahdolliset ristiintaulukoinnit ja kirjoitan ne
#Excel-tiedostoon ristit.xlsx
#Alustan ExcelWriterin kirjoittamaan uuteen tiedostoon frekvenssit.xlsx
writer = pd.ExcelWriter('ristit.xlsx', engine='xlsxwriter')

#Pidän kirjaa rivinumerosta rivi-muuttujan avulla. Python aloittaa rivinumeroinnin arvosta 0
rivi=0

#For-silmukoilla käyn läpi kaikki muuttujaparit
for var1 in df_osa:
    for var2 in df_osa:
        if var1!=var2:
            df5=pd.crosstab(df_osa[var1],df_osa[var2])
            df5.to_excel(writer, sheet_name='Ristiintaulukoinnit', startrow=rivi)
            rivi=rivi+df5.shape[0]+2

#Tallennan Excel-tiedoston
writer.save()